# FOPDT-modellering – Interaktiv Colab-notatbok

Last opp en datafil (`pid_test.txt`), juster parametre med sliders,
og last ned ferdig figur som PNG.

In [ ]:
# === Colab-oppsett (kjøres bare hvis nødvendig) ===
try:
    import ipywidgets
except ImportError:
    !pip install ipywidgets
    import ipywidgets

from google.colab import output
output.enable_custom_widget_manager()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Button, VBox, HBox, Output
from IPython.display import display, Markdown
from google.colab import files



# -----------------------------
# Last opp fil
# -----------------------------
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Pandas – automatisk separator, desimal=komma
df = pd.read_csv(filename, sep=None, engine='python', decimal=',')

tid_data = df.iloc[:,0].values
niva_data = df.iloc[:,1].values

# -----------------------------
# Automatisk estimering
# -----------------------------
y0_est = niva_data[0]
A_est = niva_data[-1] - y0_est
thresh10= y0_est + 0.1 * A_est
thresh85 = y0_est + 0.85 * A_est

L_est10 = tid_data[np.where(niva_data > thresh10)[0][0]]
L_est85 = tid_data[np.where(niva_data > thresh85)[0][0]]
L_est = abs(L_est10-0.13*(L_est85-L_est10))
thresh63 = y0_est + 0.63 * A_est
T_est = tid_data[np.where(niva_data > thresh63)[0][0]]-L_est

display(Markdown(
    f"**Autoestimat:** A={A_est:.2f}, T={T_est:.2f}, L={L_est:.2f}, y0={y0_est:.2f}"
))

# -----------------------------
# Plott for visning
# -----------------------------
def plot_fopdt_display(A, T, L, y0):
    y_model = np.where(
        tid_data < L,
        y0,
        y0 + A * (1 - np.exp(-(tid_data - L) / T))
    )

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(tid_data, niva_data, "b.", markersize=3, label="Måledata")
    ax.plot(tid_data, y_model, "r-",
            label=f"Modell: A={A:.2f}, T={T:.2f}, L={L:.2f}, y0={y0:.2f}")
    #hjelpeliner:
    #L line:
    ax.plot([0,L],[y0+1.2*A *(1 - np.exp(-1)),y0+1.2*A *(1 - np.exp(-1))],linestyle='dotted')
    #vertikal hjelpelinje for y0:
    ax.plot([7,7],[0,y0],linestyle='dotted')
    #vertikal hjelpeline for 63%
    ax.plot([T+L,T+L],[y0,y0+A],linestyle='dotted')
    #hjelpelinje for A
    ax.plot([tid_data[-1],tid_data[-1]],[y0,y0+A * (1 - np.exp(-(tid_data[-1] - L) / T))],linestyle='dotted')
    #hjelpelinje for L
    ax.plot([L,L],[0,y0+1.3*A *(1 - np.exp(-1))],linestyle='dotted')
    #horisontal hjelpelinje for 63%
    ax.plot([0,tid_data[-1]],[y0+A *(1 - np.exp(-1)),y0+A*(1 - np.exp(-1))],linestyle='dotted')
    #hjelpelinje for y0
    ax.plot([0,tid_data[-1]],[y0,y0],linestyle='dotted')
    ax.plot([L,L+T],[y0+1.2*A *(1 - np.exp(-1)),y0+1.2*A *(1 - np.exp(-1))],linestyle='dotted')
    #labels for hjelpelinjer
    ax.text(T*2,A*0.63+y0/2.7,f"$A_{{63\\%}}$={A*0.63:.2f}")
    ax.text(tid_data[-1]*0.9,A/2+y0,f"A={A:.2f}")
    ax.text(L*0.2,1.02*(y0+A*(1 - np.exp(-1))),f"$T_{{63\\%}}$={T+L:.2f}")
    ax.text(10,y0*0.85,f"y0={y0:.2f}")
    #vleger labesl utfra størrelse
    if (T/L>14):
      ax.text(L+T*0.5,y0+1.3*A *(1 - np.exp(-1)),f"T={T:.2f}",rotation=90)
      ax.text(0,y0+1.3*A *(1 - np.exp(-1)),f"L={L:.2f}")
    else:
      ax.text(L+0.5*T,y0+1.3*A *(1 - np.exp(-1)),f"T={T:.2f}",rotation=90)
      ax.text(L*0.25,y0+1.3*A *(1 - np.exp(-1)),f"L={L:.2f}",rotation=90)
    #aksetitler
    ax.set_xlabel("Tid [t]")
    ax.set_ylabel("Nivå / respons")
    ax.minorticks_on()
    ax.grid(which='both')
    ax.legend()

    plt.show()

# -----------------------------
# Plott kun for PNG
# -----------------------------
def plot_fopdt_png(A, T, L, y0):
    y_model = np.where(
        tid_data < L,
        y0,
        y0 + A * (1 - np.exp(-(tid_data - L) / T))
    )

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(tid_data, niva_data, "b.", markersize=3, label="Måledata")
    ax.plot(tid_data, y_model, "r-",
            label=f"Modell: A={A:.2f}, T={T:.2f}, L={L:.2f}, y0={y0:.2f}")
    #hjelpeliner:
    #L line:
    ax.plot([0,L],[y0+1.2*A *(1 - np.exp(-1)),y0+1.2*A *(1 - np.exp(-1))],linestyle='dotted')
    #vertikal hjelpelinje for y0:
    ax.plot([7,7],[0,y0],linestyle='dotted')
    #vertikal hjelpeline for 63%
    ax.plot([T+L,T+L],[y0,y0+A],linestyle='dotted')
    #hjelpelinje for A
    ax.plot([tid_data[-1],tid_data[-1]],[y0,y0+A * (1 - np.exp(-(tid_data[-1] - L) / T))],linestyle='dotted')
    #hjelpelinje for L
    ax.plot([L,L],[0,y0+1.3*A *(1 - np.exp(-1))],linestyle='dotted')
    #horisontal hjelpelinje for 63%
    ax.plot([0,tid_data[-1]],[y0+A *(1 - np.exp(-1)),y0+A*(1 - np.exp(-1))],linestyle='dotted')
    #hjelpelinje for y0
    ax.plot([0,tid_data[-1]],[y0,y0],linestyle='dotted')
    ax.plot([L,L+T],[y0+1.2*A *(1 - np.exp(-1)),y0+1.2*A *(1 - np.exp(-1))],linestyle='dotted')
    #labels for hjelpelinjer
    ax.text(T*2,A*0.63+y0/2.7,f"$A_{{63\\%}}$={A*0.63:.2f}")
    ax.text(tid_data[-1]*0.9,A/2+y0,f"A={A:.2f}")
    ax.text(L*0.2,1.02*(y0+A*(1 - np.exp(-1))),f"$T_{{63\\%}}$={T+L:.2f}")
    ax.text(10,y0*0.85,f"y0={y0:.2f}")
    #vleger labesl utfra størrelse
    if (T/L>14):
      ax.text(L+T*0.5,y0+1.3*A *(1 - np.exp(-1)),f"T={T:.2f}",rotation=90)
      ax.text(0,y0+1.3*A *(1 - np.exp(-1)),f"L={L:.2f}")
    else:
      ax.text(L+0.5*T,y0+1.3*A *(1 - np.exp(-1)),f"T={T:.2f}",rotation=90)
      ax.text(L*0.25,y0+1.3*A *(1 - np.exp(-1)),f"L={L:.2f}",rotation=90)
    #aksetitler
    ax.set_xlabel("Tid [t]")
    ax.set_ylabel("Nivå / respons")
    ax.minorticks_on()
    ax.grid(which='both')
    ax.legend()

    filepath = "FOPDT_plot.png"
    fig.canvas.draw()
    fig.savefig(filepath, dpi=600, bbox_inches="tight")
    plt.close(fig)
    files.download(filepath)

# -----------------------------
# Widgets
# -----------------------------
A_slider = FloatSlider(value=A_est, min=0, max=2*A_est, step=A_est/100, description="A")
T_slider = FloatSlider(value=T_est, min=1, max=2*T_est, step=0.1, description="T")
L_slider = FloatSlider(value=L_est, min=0.00001, max=8*L_est, step=0.1, description="L")
y0_slider = FloatSlider(value=y0_est, min=0, max=2*y0_est, step=A_est/100, description="y0")

save_button = Button(description="Last ned PNG")

out = Output()

# -----------------------------
# Oppdater plott når slider endres
# -----------------------------
def update_plot(change):
    with out:
        out.clear_output(wait=True)
        plot_fopdt_display(A_slider.value, T_slider.value,
                           L_slider.value, y0_slider.value)

A_slider.observe(update_plot, "value")
T_slider.observe(update_plot, "value")
L_slider.observe(update_plot, "value")
y0_slider.observe(update_plot, "value")

save_button.on_click(lambda b: plot_fopdt_png(
    A_slider.value, T_slider.value, L_slider.value, y0_slider.value
))

# Layout
ui = HBox([out, VBox([A_slider, L_slider, y0_slider, T_slider, save_button])])
display(ui)

# Første plot
update_plot(None)

# Instruksjoner
display(Markdown(
r"""
## Instruksjoner til elevene

Dra sliderne for A, T, L og y0 for å tilpasse modellen til måledataene.

Den røde kurven (modellen) oppdateres automatisk hver gang du endrer verdiene.


## Praktisk test av PID-regulator på anlegget (IMC/SIMC-tank)

Når du har tilpasset modellen og funnet rimelige verdier for
$A$, $T$, $L$ og $y_0$, kan du beregne PID-parametrene ved IMC/SIMC-metoden.

Trykk **Last ned PNG** for å lagre plottet. Les av verdiene A, T, L og y0 fra grafen.

Velg forskjellige verdier for $\lambda$, beregn parametrene og test regulatoren
på det fysiske anlegget.

### Forslag til tre tester:
"""
))



# Data
df = pd.DataFrame({
    "Valg av λ": ["λ =T/2", "λ = T/4", "λ = T/6"],
    "Forventet regulering": ["Rolig", "Standard", "Rask"],
    "Hva du bør observere": [
        "Lite oversving, tregere respons",
        "God balanse mellom fart og stabilitet",
        "Kort innreguleringstid, mulig oversving"
    ]
})
# Venstrejuster innhold og header
display(
    df.style
      .hide(axis="index")
      .set_properties(**{"text-align": "left"})       # venstrejuster celler
      .set_table_styles([                             # venstrejuster header
          {"selector": "th", "props": [("text-align", "left")]}
      ])
)

display(Markdown(
r"""

### Beregn PID-parametre (PI for tank)

Først:
- $K = \dfrac{A}{\Delta u}$

Deretter:

- $K_p = \dfrac{T}{K(\lambda + L)}$

- $T_i = min(T,4\cdot (\lambda+L))$

*(Derivatledd brukes vanligvis ikke for nivåtanker)*

---

### Fremgangsmåte i laboratoriet

1. Velg en verdi for $\lambda$.
2. Regn ut $K_p$ og $T_i$ manuelt.
3. Still inn regulatoren i LabVIEW.
4. Gjør en sprangndring i SKAL verdien (set-punkt) for nivået.
5. Sammenlign:
   - innreguleringstid
   - oversving
   - stabilitet i reguleringen

---

### Refleksjon

- Hvordan endres responsen når $\lambda$ økes?
- Hvilken $\lambda$ gir best kompromiss mellom hastighet og stabilitet?

**Merk:** Det finnes ingen «fasit».  Målet er å forstå sammenhengen
mellom modell, regulering og virkelig prosess.  Det finnes ingen «riktig» verdi for $\lambda$.
Valget er et kompromiss mellom **fart**,**stasjonæravvik** og **stabilitet**

"""
))